# Feature Selection & Regularization

In this notebook, we will see:
- Forward and backward subset selection
- L1 and L2 regularization (LASSO, ridge and elastic net)

We will apply these methods to the example from Notebook 3, using the same dataset (diabetes.csv).

In [1]:
import pandas as pd

# Load data
df = pd.read_csv('diabetes.csv', index_col = 0)
df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,59.0,2.0,32.1,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0
1,48.0,1.0,21.6,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0
2,72.0,2.0,30.5,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0
3,24.0,1.0,25.3,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0
4,50.0,1.0,23.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,2.0,28.2,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0
438,47.0,2.0,24.9,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0
439,60.0,2.0,24.9,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0
440,36.0,1.0,30.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0


We'll use the same X, and generate polynomial features up to degree 2. For simplicitly, let's do these all at once.

Recall from Notebook 3 that including all of these polynomial features resulted in *worse* performance!

Let's see if subset selection and regularization can identify the important features to keep.

In [2]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 2)

X = pd.DataFrame(data = poly.fit_transform(df[['bmi', 'bp', 's3', 's4', 's5']]),
                 columns = poly.get_feature_names_out()
                )
y = df['target']

X

,1,bmi,bp,s3,s4,s5,bmi^2,bmi bp,bmi s3,bmi s4,...,bp^2,bp s3,bp s4,bp s5,s3^2,s3 s4,s3 s5,s4^2,s4 s5,s5^2
0,1.0,32.1,101.00,38.0,4.00,4.8598,1030.41,3242.100,1219.8,128.400,...,10201.0000,3838.00,404.0000,490.839800,1444.0,152.00,184.6724,16.0000,19.439200,23.617656
1,1.0,21.6,87.00,70.0,3.00,3.8918,466.56,1879.200,1512.0,64.800,...,7569.0000,6090.00,261.0000,338.586600,4900.0,210.00,272.4260,9.0000,11.675400,15.146107
2,1.0,30.5,93.00,41.0,4.00,4.6728,930.25,2836.500,1250.5,122.000,...,8649.0000,3813.00,372.0000,434.570400,1681.0,164.00,191.5848,16.0000,18.691200,21.835060
3,1.0,25.3,84.00,40.0,5.00,4.8903,640.09,2125.200,1012.0,126.500,...,7056.0000,3360.00,420.0000,410.785200,1600.0,200.00,195.6120,25.0000,24.451500,23.915034
4,1.0,23.0,101.00,52.0,4.00,4.2905,529.00,2323.000,1196.0,92.000,...,10201.0000,5252.00,404.0000,433.340500,2704.0,208.00,223.1060,16.0000,17.162000,18.408390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,1.0,28.2,112.00,42.0,4.00,4.9836,795.24,3158.400,1184.4,112.800,...,12544.0000,4704.00,448.0000,558.163200,1764.0,168.00,209.3112,16.0000,19.934400,24.836269
438,1.0,24.9,75.00,42.0,5.00,4.4427,620.01,1867.500,1045.8,124.500,...,5625.0000,3150.00,375.0000,333.202500,1764.0,210.00,186.5934,25.0000,22.213500,19.737583
439,1.0,24.9,99.67,43.0,3.77,4.1271,620.01,2481.783,1070.7,93.873,...,9934.1089,4285.81,375.7559,411.348057,1849.0,162.11,177.4653,14.2129,15.559167,17.032954
440,1.0,30.0,95.00,42.0,4.79,5.1299,900.00,2850.000,1260.0,143.700,...,9025.0000,3990.00,455.0500,487.340500,1764.0,201.18,215.4558,22.9441,24.572221,26.315874


In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

lin = make_pipeline(
    StandardScaler(), 
    LinearRegression()
)

lin

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

Let's apply 5-fold cross validation liked we learned in Notebook 4:

In [4]:
from sklearn.model_selection import KFold, cross_validate

# Initialize cross validator
cv = KFold(n_splits = 5, shuffle = True, random_state = 23)

# Cross validate (cross_val_score computes NEGATIVE mean squared error)
results = cross_validate(lin, X, y, scoring = 'neg_mean_squared_error', cv = cv)
results = pd.DataFrame(results)
results

,fit_time,score_time,test_score
0,0.016677,0.000000,-3808.032833
1,0.008072,0.000000,-2469.683469
2,0.000000,0.000000,-3754.698790
3,0.008172,0.000000,-3269.591912
4,0.000000,0.008295,-3111.008339


In [5]:
# Extract the MSE (note the minus sign)
mse = - results['test_score']
mse

0    3808.032833
1    2469.683469
2    3754.698790
3    3269.591912
4    3111.008339
Name: test_score, dtype: float64

In [6]:
mse.mean()

3282.603068693358

# Displaying coefficients

Using ```cross_validate``` made it much easier to compute the test errors. However, if we want to look at the coefficients for the models trained on each fold, we can't: the models were not stored anywhere!

In order to extract the linear regression coefficients from each of the 5 trained models, we need to set ```return_estimator = True``` in ```cross_validate```:

In [7]:
results = cross_validate(lin, X, y, scoring = 'neg_mean_squared_error', cv = cv, return_estimator = True)  #return estimator is true if we want the corss validation to keep the model
results

{'fit_time': array([0.00803399, 0.01595449, 0.00799799, 0.        , 0.00819373]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'estimator': [Pipeline(steps=[('standardscaler', StandardScaler()),
                  ('linearregression', LinearRegression())]),
  Pipeline(steps=[('standardscaler', StandardScaler()),
                  ('linearregression', LinearRegression())]),
  Pipeline(steps=[('standardscaler', StandardScaler()),
                  ('linearregression', LinearRegression())]),
  Pipeline(steps=[('standardscaler', StandardScaler()),
                  ('linearregression', LinearRegression())]),
  Pipeline(steps=[('standardscaler', StandardScaler()),
                  ('linearregression', LinearRegression())])],
 'test_score': array([-3808.03283314, -2469.68346928, -3754.69878987, -3269.591912  ,
        -3111.00833918])}

Notice that there are 5 estimators, corresponding to the 5 folds.

Let's extract the coefficients from each of the 5 estimators, and place them in a DataFrame:

In [8]:
# Initialize an empty dataframe
coefs = pd.DataFrame(index = X.columns)  #creaitng a dataframe to store the coefficients

# Insert 1 column of coefficients for each fold
for i in range(5):
    # Select the model
    model = results['estimator'][i]  #gives the list of models
    
    # Extract the coefficients
    model_coefs = model.named_steps['linearregression'].coef_
    
    # Insert these as a new column in our dataframe
    coefs[i] = model_coefs

coefs

,0,1,2,3,4
1,0.000000,0.000000,0.000000,0.000000,0.000000
bmi,-81.866939,-45.076573,-67.205994,-21.187111,-1.053212
bp,-74.376245,-32.086037,-79.271806,-92.140973,-101.693220
s3,2.513771,48.634838,-51.045139,-73.615974,22.095463
s4,20.879524,75.890406,38.549719,-29.183725,42.933032
s5,80.800912,54.670463,-33.034527,5.731403,17.260630
bmi^2,52.214201,40.945307,42.509058,-17.443256,11.704104
bmi bp,85.624303,20.436174,72.672053,116.903540,63.611928
bmi s3,-19.512785,-6.189257,-3.453199,8.391538,-43.479465
bmi s4,-14.941477,-11.122714,-9.073300,66.830668,-40.740385


We can add some colour to make the high and low coefficients more obvious:

In [9]:
coefs.style.background_gradient(cmap = 'bwr_r', vmin = -100, vmax = 100)

,0,1,2,3,4
1,0.000000,0.000000,0.000000,0.000000,0.000000
bmi,-81.866939,-45.076573,-67.205994,-21.187111,-1.053212
bp,-74.376245,-32.086037,-79.271806,-92.140973,-101.693220
s3,2.513771,48.634838,-51.045139,-73.615974,22.095463
s4,20.879524,75.890406,38.549719,-29.183725,42.933032
s5,80.800912,54.670463,-33.034527,5.731403,17.260630
bmi^2,52.214201,40.945307,42.509058,-17.443256,11.704104
bmi bp,85.624303,20.436174,72.672053,116.903540,63.611928
bmi s3,-19.512785,-6.189257,-3.453199,8.391538,-43.479465
bmi s4,-14.941477,-11.122714,-9.073300,66.830668,-40.740385


Note that some features have consistently positive or negative coefficients across all folds, but other feature's coefficients fluctuate wildly, indicating that we are probably overfitting.

This is an indication that feature selection or regularization might be needed.

## Feature selection

The above model was trained on all features. Let's see if forward and backward selection pick out more meaningful features.

Feature selection can be very easily done using sklearn's [SequentialFeatureSelector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html).

This is a transformation step, so we can add it to our pipeline.

In [10]:
from sklearn.feature_selection import SequentialFeatureSelector

forward = SequentialFeatureSelector(lin, # <- specify the model to use 
                                    direction = 'forward', # <- set the direction
                                    n_features_to_select = 'auto',
                                    scoring = 'neg_mean_squared_error', 
                                    tol = 10, # Add feature if it improves MSE by at least this amount  #this value is adjusted according to mean squared error
                                    cv = cv
                                   )

forward.fit(X, y)
forward.get_feature_names_out()

array(['s3', 'bmi s5', 'bp s5', 's3 s4'], dtype=object)

In [11]:
backward = SequentialFeatureSelector(lin, 
                                     direction = 'backward', # <- different direction
                                     n_features_to_select = 'auto',
                                     scoring = 'neg_mean_squared_error',
                                     tol = 10, # Remove feature if it improves MSE by at least this amount
                                     cv = cv)
backward.fit(X, y)
backward.get_feature_names_out()

array(['1', 'bp', 's3', 's4', 'bmi bp', 'bmi s3', 'bmi s4', 'bp^2',
       'bp s3', 'bp s4', 'bp s5', 's3^2', 's3 s4', 's3 s5', 's5^2'],
      dtype=object)

Let's compare the MSE using these features, versus the full model above with all features.

In [12]:
X_forward  = X[forward.get_feature_names_out()]
X_backward = X[backward.get_feature_names_out()]

forward_mse = - cross_validate(lin, X_forward, y, scoring = 'neg_mean_squared_error', cv = cv)['test_score'].mean()
backward_mse = - cross_validate(lin, X_backward, y, scoring = 'neg_mean_squared_error', cv = cv)['test_score'].mean()

print('Full model:\t', mse.mean())
print('Forward:\t', forward_mse)
print('Backward:\t', backward_mse)

Full model:	 3282.603068693358
Forward:	 3022.6080465738683
Backward:	 3024.365924049757


Both forward and backward selection:
- have picked out a subset of all features
- have improved MSE by quite a bit

Both forward and backward selection have similar MSE, but forward selection achieved this with much fewer features!

Try with different values of ```tol```, or set ```n_features_to_select``` to a fixed number instead.


## LASSO regression

Let's now see what happens when we use regularization instead of feature selection.

Let's start with [LASSO](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), with ```alpha = 1``` (recall that ```alpha``` is the same as $\lambda$)

The following code is mostly a repeat of what we've seen, but with LASSO instead of Linear Regression.

In [13]:
from sklearn.linear_model import Lasso

# Initialize model (LASSO)
lasso = make_pipeline(StandardScaler(), Lasso(alpha = 1))

In [14]:
# Cross validate
results = cross_validate(lasso, # <- lasso instead of lin
                         X, y, scoring = 'neg_mean_squared_error', cv = cv, return_estimator = True)

mse = - results['test_score']
mse.mean()

3029.4666319113794

We have also managed to improve the performance! Let's look at the coefficients:

In [15]:
# Display coefficients in a nice DataFrame
coefs = pd.DataFrame(index = X.columns)

for i in range(5):
    model = results['estimator'][i]
    coefs[i] = model.named_steps['lasso'].coef_ # remember to specify the correct step in the pipeline

coefs

,0,1,2,3,4
1,0.000000,0.000000,0.000000,0.000000,0.000000
bmi,0.000000,0.000000,0.000000,0.000000,0.000000
bp,0.000000,0.000000,0.000000,0.000000,0.000000
s3,-0.000000,-4.439389,-6.706802,-7.623467,-4.289952
s4,0.000000,0.000000,0.000000,0.000000,0.000000
s5,23.770583,10.449538,21.204453,18.032042,16.019595
bmi^2,13.447530,7.951429,8.104222,0.000000,0.000000
bmi bp,22.084885,10.027020,21.600845,29.417419,24.362847
bmi s3,-6.316139,-0.000000,-0.000000,-0.000000,-3.771338
bmi s4,2.370266,0.000000,0.000000,0.033388,0.000000


We see that many of the coefficients are now zero! Let's remove the non-zero entries and colour them:

In [16]:
coefs[(coefs != 0).sum(axis = 1) > 0].style.background_gradient(cmap = 'bwr_r', vmin = -100, vmax = 100)

,0,1,2,3,4
s3,-0.000000,-4.439389,-6.706802,-7.623467,-4.289952
s5,23.770583,10.449538,21.204453,18.032042,16.019595
bmi^2,13.447530,7.951429,8.104222,0.000000,0.000000
bmi bp,22.084885,10.027020,21.600845,29.417419,24.362847
bmi s3,-6.316139,-0.000000,-0.000000,-0.000000,-3.771338
bmi s4,2.370266,0.000000,0.000000,0.033388,0.000000
bmi s5,2.330237,20.747560,8.473513,8.760257,17.871233
bp s5,0.000000,8.131265,0.000000,0.000000,0.175217
s3^2,-0.000000,-3.586836,-0.000000,-0.000000,-0.000000
s3 s4,-4.360892,-3.655541,-3.983023,-2.501566,-4.385367


Compare these to the coefficients we saw in forward and backward selection.

In summary, by doing LASSO:
- Coefficients are generally smaller, and fluctuate less between folds
- Many coefficients are 0
- MSE has improved

Note that this was done with a single ```alpha = 1```. In practice, we usually try a few alphas ranging from 0.0001, 0.001, ... 1, 10, 1000, and then pick the best one.

## Ridge regression

Let's try this with [Ridge regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)

In [17]:
from sklearn.linear_model import Ridge

# Initialize model (Ridge)
ridge = make_pipeline(StandardScaler(), Ridge(alpha = 1))

In [18]:
# Cross validate
results = cross_validate(ridge, X, y, scoring = 'neg_mean_squared_error', cv = cv, return_estimator = True)

mse = - results['test_score']
mse.mean()

3088.2594739569568

In [19]:
# Display coefficients in a nice DataFrame
coefs = pd.DataFrame(index = X.columns)

for i in range(5):
    model = results['estimator'][i]
    coefs[i] = model.named_steps['ridge'].coef_ # remember to specify the correct step

coefs[(coefs != 0).sum(axis = 1) > 0].style.background_gradient(cmap = 'bwr_r', vmin = -100, vmax = 100)

,0,1,2,3,4
bmi,-33.815122,-17.912446,-31.001823,-12.154190,-10.858104
bp,-36.088006,-16.816606,-40.854075,-40.152649,-43.420708
s3,-2.146586,1.651688,-20.585915,-19.374191,-0.718675
s4,4.412488,18.255620,19.046238,2.021466,14.800388
s5,31.947353,27.241657,-2.990006,4.809322,14.574786
bmi^2,31.475582,28.177299,29.246458,-2.990870,15.049996
bmi bp,55.064994,15.652894,47.898574,64.005277,42.443329
bmi s3,-24.305897,-9.437261,-7.805394,-5.978217,-19.486503
bmi s4,-1.402228,0.000498,1.111410,29.559278,-4.414160
bmi s5,13.773030,21.070406,5.230250,-12.013869,13.092485


Compare this to the coefficients for LASSO. Note that:

- Coefficients are smaller than the unregularized case
- However, many of them remain non-zero
- MSE has improved, though not as much as lasso

## Elastic Net

Finally, we use Elastic Net. Instead of $\lambda_1$ and $\lambda_2$, sklearn's Elastic Net has ```alpha``` and ```l1_ratio```.

Let's use ```l1_ratio = 0.5``` which places equal importance on the l1 and l2 penalties.

In [20]:
from sklearn.linear_model import ElasticNet

elastic =  make_pipeline(StandardScaler(), ElasticNet(alpha = 1, l1_ratio = 0.5))

In [21]:
# Cross validate
results = cross_validate(elastic, X, y, scoring = 'neg_mean_squared_error', cv = cv, return_estimator = True)

mse = - results['test_score']
mse.mean()

3054.585194900448

In [22]:
# Display coefficients in a nice DataFrame
coefs = pd.DataFrame(index = X.columns)

for i in range(5):
    model = results['estimator'][i]
    coefs[i] = model.named_steps['elasticnet'].coef_ # remember to specify the correct step!

coefs[(coefs != 0).sum(axis = 1) > 0].style.background_gradient(cmap = 'bwr_r', vmin = -100, vmax = 100)

,0,1,2,3,4
bmi,5.645599,6.805795,6.366060,5.774095,5.816832
bp,2.076964,2.391189,2.069807,3.339551,2.622117
s3,-2.552644,-3.183017,-2.843448,-2.898249,-2.970336
s4,0.120324,0.000000,0.000000,0.000000,0.000000
s5,7.079245,6.807562,7.146600,6.041671,6.674847
bmi^2,6.947498,7.694962,7.457524,6.261882,6.579938
bmi bp,6.620542,6.435507,6.913389,7.100866,6.596009
bmi s4,3.475683,2.770564,2.669477,2.903973,2.999932
bmi s5,7.886115,8.496797,8.313484,7.098137,7.827845
bp^2,2.908898,2.701184,2.721377,4.094017,3.319238


# Summary

In summary, we've seen that feature selection, Lasso, Ridge and Elastic Net all have improved our model, but in different ways (they all ended up with different feature coefficients).

In this particular case, feature selection and Lasso worked quite well. But results might differ on other datasets.

Note that we haven't even tried varying the regularization parameters! We'll see how to do this next time!